<a href="https://colab.research.google.com/github/Yucheol-Son-BYUI/CSE310_W0_HelloWorld/blob/main/notebooks/starter_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [40]:
df.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,37069.000000,37069.000000,37069.000000,37069.000000,37069.000000,37069.000000,37069.000000,37069.000000,37069.000000
mean,40.025493,2.564407,962.221803,0.173730,0.081526,93.576551,-40.494829,3.621945,5167.010650
std,10.435288,2.764084,187.531477,0.496159,1.572287,0.579339,4.628895,1.734496,72.294476
min,17.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [41]:
df.describe(exclude='number').columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome', 'y'],
      dtype='object')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37069 entries, 0 to 37068
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37069 non-null  int64  
 1   job             37069 non-null  object 
 2   marital         37069 non-null  object 
 3   education       37069 non-null  object 
 4   default         37069 non-null  object 
 5   housing         37069 non-null  object 
 6   loan            37069 non-null  object 
 7   contact         37069 non-null  object 
 8   month           37069 non-null  object 
 9   day_of_week     37069 non-null  object 
 10  campaign        37069 non-null  int64  
 11  pdays           37069 non-null  int64  
 12  previous        37069 non-null  int64  
 13  poutcome        37069 non-null  object 
 14  emp.var.rate    37069 non-null  float64
 15  cons.price.idx  37069 non-null  float64
 16  cons.conf.idx   37069 non-null  float64
 17  euribor3m       37069 non-null 

1. Since 88% of our target labels are 'no,' we're dealing with a class imbalance. We should prioritize the F1-score over Accuracy.

2. At least 75% of the pdays values are '999.' This definitely needs special handling since it's a placeholder, not a literal number.

3. Should we clip the outliers in the campaign column, like that maximum value of 56?

4. The default column is overwhelmingly 'no' (29,341 cases). We need to figure out if it even provides enough information for the model.

5. We should normalize our quantitative features, especially if we're planning to use distance-based models.

In [46]:
## Pdays
# 1. Create the 'is_new_customer' binary flag
df['is_new_customer'] = (df['pdays'] == 999).astype(int)

# 2. Replace the placeholder original 999 with -1
df['pdays'] = df['pdays'].replace(999, -1)

# print(df[['pdays', 'is_new_customer']].head(10))

## education as ordinal data
edu_mapping = {
    'illiterate': 0,
    'unknown': 1,  # You can also use -1 here
    'basic.4y': 2,
    'basic.6y': 3,
    'basic.9y': 4,
    'high.school': 5,
    'professional.course': 6,
    'university.degree': 7
}

df['education'] = df['education'].map(edu_mapping)

## Other Categorical Data
## One-hot encoding
categorical_cols = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

df_final = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_final['y'] = df_final['y'].map({'yes': 1, 'no': 0})


In [47]:
df_final.shape

(37069, 48)

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score

# 1. Preprocess data
# already done above

# 2. Prepare training data
X = df_final.drop('y', axis=1)
y = df_final['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Training
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1 # 모든 CPU 코어 사용
)
rf_model.fit(X_train, y_train)

# 4. Threadshold optimization
y_probs = rf_model.predict_proba(X_test)[:, 1]
custom_threshold = 0.51
y_pred_custom = (y_probs >= custom_threshold).astype(int)

# 5. Report
print(f"--- RandomForest Results (Threshold: {custom_threshold}) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_custom):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_custom):.4f}")
print("\n[Full Classification Report]")
print(classification_report(y_test, y_pred_custom))

importances = pd.Series(rf_model.feature_importances_, index=X.columns)
print("\n[Top 10 Feature Importances]")
print(importances.sort_values(ascending=False).head(10))

--- RandomForest Results (Threshold: 0.51) ---
Accuracy: 0.8824
F1 Score: 0.5118

[Full Classification Report]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      6572
           1       0.48      0.54      0.51       842

    accuracy                           0.88      7414
   macro avg       0.71      0.73      0.72      7414
weighted avg       0.89      0.88      0.89      7414


[Top 10 Feature Importances]
euribor3m            0.164051
nr.employed          0.128631
age                  0.094535
cons.conf.idx        0.064655
emp.var.rate         0.063668
campaign             0.052315
pdays                0.045100
cons.price.idx       0.038050
poutcome_success     0.027211
contact_telephone    0.024504
dtype: float64
